In [1]:
library(R.matlab)
library(wavelets)
library(tuneR)
library(seewave)
library(data.table)
library(constellation)
library(GENEAread)

R.matlab v3.6.2 (2018-09-26) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen


Loading required package: bitops

Loading required package: mmap



In [ ]:
baseName <- "frame"
wavesName = c("8PSK", "16QAM", "64QAM", "BPSK", "CPFSK", "GFSK", "QPSK", "PAM4")
dffile <- data.frame(modulation=wavesName)

timeDomain_columns <- paste('timeDomain', c(1:1024), sep='_')
dffile[, timeDomain_columns] <- NA

fourierDomain_columns <- paste('fourierDomain', c(1:1024), sep='_')
dffile[, fourierDomain_columns] <- NA

waveletsDomain_columns <- c("wavelets_max_serie_1", "wavelets_min_serie_1" ,"wavelets_mean_serie_1", "wavelets_1st_Qu_serie_1" , "wavelets_2nd_Qu_serie_1", "wavelets_3rd_Qu_serie_1", 
                            "wavelets_max_serie_2", "wavelets_min_serie_2" ,"wavelets_mean_serie_2", "wavelets_1st_Qu_serie_2" , "wavelets_2nd_Qu_serie_2", "wavelets_3rd_Qu_serie_2", 
                            "wavelets_max_serie_3", "wavelets_min_serie_3" ,"wavelets_mean_serie_3", "wavelets_1st_Qu_serie_3" , "wavelets_2nd_Qu_serie_3", "wavelets_3rd_Qu_serie_3") 
dffile[, waveletsDomain_columns] <- NA

zeroCrossX_columns <- paste('zeroCross_X_', c(1:1024), sep='_')
dffile[, zeroCrossX_columns] <- NA

zeroCrossY_columns <- paste('zeroCross_Y_', c(1:1024), sep='_')
dffile[, zeroCrossY_columns] <- NA

autoCorrelation_columns <- paste('autoCorrelation_', c(1:1024), sep='_')
dffile[, autoCorrelation_columns] <- NA

cs_current_columns <- paste('cs_current_', c(1:1024), sep='_')
dffile[, cs_current_columns] <- NA

cs_prior_columns <- paste('cs_prior_', c(1:1024), sep='_')
dffile[, cs_prior_columns] <- NA

s_ww_columns <- c("sDomain_max", "sDomain_min" ,"sDomain_mean", "sDomain_1st_Qu" , "sDomain_2nd_Qu", "sDomain_3rd_Qu")
dffile[, s_ww_columns] <- NA
names_idx=1
row_idx = 1

for(i in 1:1000) {
    for (w in wavesName) {
        n <- formatC(i, width=3, flag="0")
        name <- paste(baseName, w, n, ".mat", sep="")
        pathfile <- paste("../dataset/origin/", name, sep="")
        signal <- readMat(pathfile)
        values <- signal['frame']$frame
        
        if(i > 1){
            dffile[row_idx, 'modulation'] <- wavesName[names_idx] 
            names_idx = names_idx + 1
            if(names_idx == 8) names_idx=1
        }
        
        #Time Domain
        t=seq(0,1023)
        s = Mod(values) * cos((pi*2.5*t) + Arg(values))
        dffile[row_idx, timeDomain_columns] <- s
        
        
        #Frequency Domain (Fourier)
        f_s <- fft(s)
        f_s_abs <- abs(f_s)
        dffile[row_idx, fourierDomain_columns] <- f_s_abs
        
        #Frequency Domain (Wavelets)
        s_wavelets <- dwt(s, filter="la8", n.levels=3, boundary="periodic", fast=TRUE)
        s_ww <- c(max(s_wavelets@W$W1), min(s_wavelets@W$W1), mean(s_wavelets@W$W1), quantile(s_wavelets@W$W1, probs = c(0.25, 0.5, 0.75), names=FALSE),
                   max(s_wavelets@W$W2), min(s_wavelets@W$W2), mean(s_wavelets@W$W2), quantile(s_wavelets@W$W2, probs = c(0.25, 0.5, 0.75), names=FALSE),
                   max(s_wavelets@W$W3), min(s_wavelets@W$W3), mean(s_wavelets@W$W3), quantile(s_wavelets@W$W3, probs = c(0.25, 0.5, 0.75), names=FALSE))
        dffile[row_idx, waveletsDomain_columns] <- s_ww
        
        #Zero-Cossing
        s_wave <- Wave(left=s, samp.rate=2500000000, bit=8)
        s_zc <- zc(s_wave, plot=FALSE)
        s_zc_x <- s_zc[,1]
        s_zc_y <- s_zc[,2]
        
        dffile[row_idx, zeroCrossX_columns] <- s_zc_x
        dffile[row_idx, zeroCrossY_columns] <- s_zc_y
        
        #autocorrelation
        ac <- acf(s, lag = 1023, plot=FALSE)
        dffile[row_idx, autoCorrelation_columns] <- ac$acf
        
        #Constellation Shape Features
        tt <- seq(from = as.POSIXct("00:00:00", format="%H:%M:%S", tz="UTC"), length.out = 1024, by = "mins")
        DF = data.table( ID = 1, TIME = tt, VALUE = s)
        DT_C_SHAPE <- value_change(DF, value = 1.5, direction = "up", window_hours = 18, join_key = "ID", time_var = "TIME", value_var = "VALUE.V1", mult = "all")
        cs_current <- DT_C_SHAPE$CURRENT_VALUE[c(1:1024)]
        cs_prior <- DT_C_SHAPE$PRIOR_VALUE[c(1:1024)]
        dffile[row_idx, cs_current_columns] <- cs_current
        dffile[row_idx, cs_prior_columns] <- cs_prior
        
        
        #S Transform
        s_s <- stft(s, plot = FALSE, reassign = TRUE, win = 100)
        s_s_vector <- as.vector(s_s[[1]])
        s_s_statista <- c(max(s_s_vector), min(s_s_vector), mean(s_s_vector), quantile(s_s_vector, probs = c(0.25, 0.5, 0.75), names=FALSE))
        dffile[row_idx, s_ww_columns] <- s_s_statista
        
        row_idx <- row_idx + 1 
    }
}
dffile[is.na(data)] <- 0
write.csv(dffile, "../dataset/transform/data.csv", row.names = FALSE)

In [86]:
dfData <- read.csv("../dataset/transform/data100.csv")
dfData

modulation,timeDomain_1,timeDomain_2,timeDomain_3,timeDomain_4,timeDomain_5,timeDomain_6,timeDomain_7,timeDomain_8,timeDomain_9,⋯,cs_prior__1021,cs_prior__1022,cs_prior__1023,cs_prior__1024,sDomain_max,sDomain_min,sDomain_mean,sDomain_1st_Qu,sDomain_2nd_Qu,sDomain_3rd_Qu
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8PSK,0.6392886,-0.213477814,-0.86562140,0.3868681,0.621987409,-0.5673602,0.010029805,0.373295383,-0.15027261,⋯,-0.9395049,-0.9373417,-1.1584460,-1.1512771,38.69096,0.0375952649,5.076279,0.9559093,1.6212373,7.163684
16QAM,-0.3431013,-0.515627590,-0.67285437,0.5470399,1.419238555,-0.1672717,-1.138051140,-0.069712734,1.46453253,⋯,-1.6207448,-1.1815026,-1.1340259,-0.5156276,47.53911,0.0008458841,5.453533,1.8936317,3.1432403,5.845227
64QAM,-1.0545999,-0.523152444,0.88641973,0.5469051,-0.485738118,-0.5248770,-0.020109388,0.228227906,0.19388036,⋯,-0.5909732,-1.0655638,-1.2184446,-0.7388383,38.20069,0.0003831160,4.535727,0.3255653,0.5912885,6.332244
BPSK,0.2018006,0.990348264,-0.08351802,-0.9478079,0.114775866,0.9908187,-0.046716765,-1.288954028,0.39295448,⋯,-0.7546626,-0.7379081,-0.8062544,-0.6961602,37.16356,0.0066318804,4.472471,0.4133859,0.7684638,5.450173
CPFSK,0.1682005,0.239789525,-0.02312236,-0.1873929,-0.080767905,0.2081508,0.165164163,-0.273555480,-0.23753323,⋯,-1.3007446,-1.3950095,-1.4106632,-1.3835652,59.12577,0.0562527140,4.774119,0.9935904,1.8385517,4.677182
GFSK,0.8969871,-1.174058031,-0.20708120,1.1995020,-0.499289947,-0.9263796,1.152250876,0.201145695,-1.28342687,⋯,-1.1736950,-0.7912100,-0.7090256,-0.7662139,49.05288,0.0225418195,4.925958,0.4659396,1.7317731,7.570201
QPSK,-0.6294614,0.612358001,0.35385368,-1.0192615,-0.003453777,1.2864124,-0.741523281,-1.127772298,1.46860999,⋯,-0.8364791,-0.8226342,-0.9361303,-1.9114611,45.46900,0.0272175949,4.767523,1.0842809,1.8665542,3.759132
PAM4,-0.9360456,1.056787447,0.95703316,-0.8039921,-0.571378881,0.4213381,0.278995700,-0.086925956,-0.03241157,⋯,-0.2283899,-0.3814172,-0.2337760,-0.2878606,37.20031,0.0138664802,4.822406,0.7110661,1.2425726,5.579549
8PSK,-0.8462353,0.963618680,0.55142095,-0.8734569,-0.419782304,0.7030569,0.447613483,-0.191146721,-0.55542440,⋯,-0.5218370,-1.1830895,-0.7630471,-0.8446055,35.71186,0.0330078661,4.789509,0.6416310,1.1238116,5.997706
